<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">MODELOS DE RECOMENDACIÓN PARA EL DESARROLLO DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [ ]:
#Conexión de Google Colab con Drive.
from google.colab import drive
drive.mount('/content/drive')

#Acceder a los archivos de la carpeta marketing para el uso de funciones.
import sys
sys.path.append('/content/drive/MyDrive/marketing/Marketing')

Mounted at /content/drive


In [ ]:
import pandas as pd # Manipulación y el analísis de datos.
import sqlite3 as sql # Interactuar con base de datos SQL usando Python.

import matplotlib.pyplot as plt
import seaborn as sns

#### CONEXIÓN CON LA BASE DE DATOS

In [ ]:
#Establece una conexión a la base de datos SQLite ubicada en "content/db_movies"
conexion = sql.connect('/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db')

# Crear un cursor a partir de la conexión, que permita ejecutar comando SQL sobre la base de datos.
cur = conexion.cursor()

In [ ]:
# Ejecuta una consulta SQL que obtiene los nombres de todas las tablas existentes en la base de datos.
cur.execute("SELECT name FROM sqlite_master where type='table' ")

# Recupera todos los resultados de la consulta anterior en forma de lista de tuplas.
cur.fetchall()

[('ratings',), ('movies',), ('ratings_movies',)]

In [ ]:
table_movies = pd.read_sql_query("SELECT * FROM movies", conexion)
table_movies.head()

,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,Toy Story,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,Jumanji,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1995,Grumpier Old Men,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,1995,Waiting to Exhale,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,1995,Father of the Bride Part II,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
table_ratings = pd.read_sql_query("SELECT * FROM ratings", conexion)
table_ratings.head()

,userId,movieId,rating,day,month,hour,year
0,1,1,4.0,30,7,18,2000
1,1,3,4.0,30,7,18,2000
2,1,6,4.0,30,7,18,2000
3,1,47,5.0,30,7,19,2000
4,1,50,5.0,30,7,18,2000


In [ ]:
ratings_movies = pd.read_sql_query("SELECT * FROM ratings_movies", conexion)
ratings_movies.head()


,userId,rating,day,month,hour,year,movieId,año,titulo,Action,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,4.0,30,7,18,2000,1,1995,Toy Story,0,...,0,0,0,0,0,0,0,0,0,0
1,1,4.0,30,7,18,2000,3,1995,Grumpier Old Men,0,...,0,0,0,0,0,1,0,0,0,0
2,1,4.0,30,7,18,2000,6,1995,Heat,1,...,0,0,0,0,0,0,0,1,0,0
3,1,5.0,30,7,19,2000,47,1995,Seven (a.k.a. Se7en),0,...,0,0,0,0,1,0,0,1,0,0
4,1,5.0,30,7,18,2000,50,1995,"Usual Suspects, The",0,...,0,0,0,0,1,0,0,1,0,0


####**SISTEMAS DE RECOMENDACION BASADA EN POPULARIDAD**
-------

Para el sistema de recomendación por popularidad, se utilizaron consultas SQL para identificar las películas con mayor número de calificaciones y mejor puntuación promedio. Se aplicaron filtros para asegurar un mínimo de votos por película y se representaron los resultados.

In [ ]:
import pandas as pd

query = """
SELECT
    m.titulo AS titulo_pelicula,
    COUNT(*) AS total_visualizaciones
FROM ratings r
JOIN movies m ON r.movieId = m.movieId
GROUP BY m.movieId
ORDER BY total_visualizaciones DESC
LIMIT 6;
"""

populares = pd.read_sql(query, conexion)
display(populares)

,titulo_pelicula,total_visualizaciones
0,Forrest Gump,329
1,"Shawshank Redemption, The",317
2,Pulp Fiction,307
3,"Silence of the Lambs, The",279
4,"Matrix, The",278
5,Star Wars: Episode IV - A New Hope,251


####**PELICULAS MEJOR CALIFICADAS CON MAS DE 30 RATINGS**

In [ ]:
query = """
SELECT m.titulo,
       AVG(r.rating) AS avg_rating,
       COUNT(*) AS num_ratings,año
FROM ratings r
JOIN movies m ON r.movieId = m.movieId
WHERE r.rating <> 0
GROUP BY m.titulo
HAVING num_ratings > 50
ORDER BY num_ratings DESC, avg_rating > 0 DESC
LIMIT 10;
"""

pd.read_sql_query(query, conexion)



,titulo,avg_rating,num_ratings,año
0,Forrest Gump,4.164134,329,1994
1,"Shawshank Redemption, The",4.429022,317,1994
2,Pulp Fiction,4.197068,307,1994
3,"Silence of the Lambs, The",4.161290,279,1991
4,"Matrix, The",4.192446,278,1999
5,Star Wars: Episode IV - A New Hope,4.231076,251,1977
6,Jurassic Park,3.750000,238,1993
7,Braveheart,4.031646,237,1995
8,Terminator 2: Judgment Day,3.970982,224,1991
9,Schindler's List,4.225000,220,1993


####**PELICULAS RECIENTES MEJOR CALIFICADAS**

In [ ]:
query = """SELECT m.titulo, m.año, AVG(r.rating) AS avg_rating, COUNT(*) AS num_ratings
FROM ratings r
JOIN movies m ON r.movieId = m.movieId
WHERE r.rating > 0
GROUP BY m.movieId
HAVING num_ratings > 50
ORDER BY m.año DESC, avg_rating DESC
LIMIT 10;
"""
pd.read_sql_query(query, conexion)


,titulo,año,avg_rating,num_ratings
0,Deadpool,2016,3.833333,54
1,Guardians of the Galaxy,2014,4.050847,59
2,Interstellar,2014,3.993151,73
3,"Grand Budapest Hotel, The",2014,3.778846,52
4,"Wolf of Wall Street, The",2013,3.916667,54
5,"Dark Knight Rises, The",2012,3.993421,76
6,Django Unchained,2012,3.943662,71
7,"Avengers, The",2012,3.869565,69
8,The Hunger Games,2012,3.435185,54
9,Toy Story 3,2010,4.109091,55


###**SISTEMA DE RECOMENDACION BASADA EN CONTENIDO O SIMILITUD**
--------

La recomendación por similitud de contenido se desarrolló a partir de los géneros binarizados de cada película. Con ayuda de sklearn, se calculó una matriz de similitud entre títulos usando la métrica de coseno. Esto permitió construir un sistema que identifica qué tan parecidos son dos títulos con base en sus características,  y con esto ofrecer recomendaciones más contextualizadas.

In [ ]:
query_generos = """
SELECT movieId, titulo,
       Action, Comedy, Drama, Horror, Romance, Thriller,
       Adventure, Animation, Children, Documentary,
       Fantasy, IMAX, Musical, Mystery, War, Western, "Film-Noir"
FROM movies;
"""

peliculas_df = pd.read_sql_query(query_generos, conexion)
peliculas_df.set_index('movieId', inplace=True)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Extraer solo columnas de géneros
generos = peliculas_df.drop(columns=['titulo'])

# Calcular matriz de similitud
similitud = cosine_similarity(generos)

# Convertir a DataFrame
import numpy as np
similitud_df = pd.DataFrame(similitud, index=peliculas_df.index, columns=peliculas_df.index)


In [ ]:
def recomendar_peliculas(movie_title, peliculas_df, similitud_df, top_n=4):
    # Buscar el ID de la película
    pelicula_id = peliculas_df[peliculas_df['titulo'].str.lower() == movie_title.lower()].index

    if pelicula_id.empty:
        print("Película no encontrada.")
        return

    pelicula_id = pelicula_id[0]

    # Obtener las similitudes
    similares = similitud_df[pelicula_id].sort_values(ascending=False)

    # Excluir la misma película
    similares = similares.drop(pelicula_id)

    # Obtener títulos de las películas similares
    recomendadas = peliculas_df.loc[similares.head(top_n).index, 'titulo']

    print(f"\n🎬 Películas similares a: {peliculas_df.loc[pelicula_id, 'titulo']}\n")
    for i, titulo in enumerate(recomendadas, 1):
        print(f"{i}. {titulo}")


In [ ]:
recomendar_peliculas("Forrest Gump", peliculas_df, similitud_df)



🎬 Películas similares a: Forrest Gump

1. Life Is Beautiful (La Vita è bella)
2. I Served the King of England (Obsluhoval jsem anglického krále)
3. Train of Life (Train de vie)
4. Tiger and the Snow, The (La tigre e la neve)


##**SISTEMA DE RECOMENDACION COLABORATIVA**
-------

In [ ]:
import pandas as pd

# Leer los datos de ratings (si no están cargados ya)
ratings_query = """
SELECT * FROM ratings;
"""
ratings_df = pd.read_sql_query(ratings_query, conexion)

# Crear la matriz de usuario vs película
ratings_matrix = ratings_df.pivot_table(index='userId', columns='movieId', values='rating')

# Ver la matriz de calificaciones
ratings_matrix.head()


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Rellenar valores nulos con la media de las calificaciones de cada película
ratings_matrix_filled = ratings_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)

# Ver la matriz con valores completados
ratings_matrix_filled.head(8)


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.366379,4.000000,4.366379,4.366379,4.000000,4.366379,4.366379,4.366379,4.366379,...,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379
2,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,...,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276
3,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,...,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897
4,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,...,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556
5,4.000000,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,...,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364
6,3.493631,4.000000,5.000000,3.000000,5.000000,4.000000,4.000000,3.000000,3.493631,3.000000,...,3.493631,3.493631,3.493631,3.493631,3.493631,3.493631,3.493631,3.493631,3.493631,3.493631
7,4.500000,3.230263,3.230263,3.230263,3.230263,3.230263,3.230263,3.230263,3.230263,3.230263,...,3.230263,3.230263,3.230263,3.230263,3.230263,3.230263,3.230263,3.230263,3.230263,3.230263
8,3.574468,4.000000,3.574468,3.574468,3.574468,3.574468,3.574468,3.574468,3.574468,2.000000,...,3.574468,3.574468,3.574468,3.574468,3.574468,3.574468,3.574468,3.574468,3.574468,3.574468


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud entre usuarios
user_similarity = cosine_similarity(ratings_matrix_filled)

# Convertir la matriz de similitudes a un DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=ratings_matrix_filled.index, columns=ratings_matrix_filled.index)

# Ver las primeras similitudes
user_similarity_df.head(8)


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.999540,0.998159,0.998163,0.999446,0.998587,0.998273,0.999449,0.999261,0.998668,...,0.999552,0.999009,0.994340,0.999343,0.998831,0.997377,0.999031,0.994773,0.999559,0.996018
2,0.999540,1.000000,0.998498,0.998417,0.999780,0.998973,0.998616,0.999754,0.999586,0.999020,...,0.999880,0.999354,0.994720,0.999706,0.999202,0.997698,0.999308,0.994891,0.999899,0.996354
3,0.998159,0.998498,1.000000,0.997014,0.998363,0.997614,0.997242,0.998350,0.998205,0.997638,...,0.998501,0.997955,0.993444,0.998324,0.997818,0.996180,0.997920,0.993453,0.998523,0.995043
4,0.998163,0.998417,0.997014,1.000000,0.998293,0.997565,0.997335,0.998314,0.998125,0.997689,...,0.998411,0.998031,0.993432,0.998237,0.997835,0.996319,0.997892,0.993245,0.998444,0.994878
5,0.999446,0.999780,0.998363,0.998293,1.000000,0.998886,0.998527,0.999656,0.999482,0.998888,...,0.999785,0.999269,0.994668,0.999606,0.999072,0.997621,0.999232,0.994790,0.999814,0.996226
6,0.998587,0.998973,0.997614,0.997565,0.998886,1.000000,0.997737,0.998806,0.998690,0.998167,...,0.998982,0.998466,0.993764,0.998778,0.998334,0.996772,0.998490,0.994101,0.999028,0.995473
7,0.998273,0.998616,0.997242,0.997335,0.998527,0.997737,1.000000,0.998536,0.998417,0.997709,...,0.998609,0.998196,0.993663,0.998470,0.998159,0.996535,0.998115,0.993792,0.998650,0.995375
8,0.999449,0.999754,0.998350,0.998314,0.999656,0.998806,0.998536,1.000000,0.999471,0.998892,...,0.999786,0.999330,0.994655,0.999667,0.999071,0.997606,0.999233,0.994917,0.999801,0.996243


In [ ]:
def recomendar_peliculas_collaborative(user_id, ratings_matrix, user_similarity_df, top_n=8):
    # Obtener las películas que el usuario ya ha calificado
    rated_movies = ratings_matrix.loc[user_id].dropna().index

    # Obtener usuarios más similares
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).drop(user_id)

    # Tomar los usuarios más similares
    most_similar_users = similar_users.head(top_n).index

    # Recomendaciones basadas en los usuarios más similares
    recommended_movies = []
    for user in most_similar_users:
        movies_rated_by_similar_user = ratings_matrix.loc[user].dropna().index
        for movie in movies_rated_by_similar_user:
            if movie not in rated_movies:
                recommended_movies.append(movie)

    # Obtener los títulos de las películas recomendadas
    recommended_movie_titles = peliculas_df.loc[recommended_movies, 'titulo'].dropna()

    print(f"\n🎥 Películas recomendadas para el usuario {user_id}:\n")
    for i, movie in enumerate(recommended_movie_titles.head(top_n), 1):
        print(f"{i}. {movie}")


In [ ]:
recomendar_peliculas_collaborative(414, ratings_matrix, user_similarity_df)



🎥 Películas recomendadas para el usuario 414:

1. Ace Ventura: When Nature Calls
2. From Dusk Till Dawn
3. Robin Hood: Men in Tights
4. Dead Man
5. Psycho
6. Seventh Seal, The (Sjunde inseglet, Det)
7. Carrie
8. Rosemary's Baby


In [ ]:
###SISTEMA DE RECOMENDACION BASADA EN EL PERFIL DEL USUARIO

In [ ]:
user_id = 414  # Puedes cambiar este ID por el que necesites

query = """
SELECT m.*
FROM ratings r
JOIN movies m ON r.movieId = m.movieId
WHERE r.userId = ? AND r.rating >= 4.0
"""
pelis_usuario = pd.read_sql_query(query, conexion, params=(user_id,))


In [ ]:
generos = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
           'Documentary', 'Drama', 'Fantasy', "Film-Noir", 'Horror', 'IMAX',
           'Musical', 'Mystery', 'Romance', "Sci-Fi", 'Thriller', 'War', 'Western']

perfil_usuario = pelis_usuario[generos].sum().sort_values(ascending=False)
display(perfil_usuario.head(5))  # Mostrar los géneros más frecuentes


,0
Action,22
Adventure,18
Sci-Fi,14
IMAX,8
Crime,7


In [ ]:
query = """
SELECT *
FROM movies
WHERE movieId NOT IN (
    SELECT movieId FROM ratings WHERE userId = ?
)
"""
pelis_no_vistas = pd.read_sql_query(query, conexion, params=(user_id,))


In [ ]:
# Multiplicamos cada fila por el perfil del usuario (ponderación por preferencia)
pelis_no_vistas['score'] = pelis_no_vistas[generos].dot(perfil_usuario)

# Ordenamos según el score
recomendaciones = pelis_no_vistas.sort_values(by='score', ascending=False)

# Mostramos las top 10 recomendaciones
display(recomendaciones[['movieId', 'titulo']].head(7))


,movieId,titulo
7526,85261,Mars Needs Moms
7147,71999,Aelita: The Queen of Mars (Aelita)
9302,161594,Kingsglaive: Final Fantasy XV
6445,52462,Aqua Teen Hunger Force Colon Movie Film for Th...
6859,62956,Futurama: Bender's Game
9338,164226,Maximum Ride
9229,157865,Ratchet & Clank




```
# Objetivo:
Agregar una capa de explicación a las recomendaciones, para que el usuario entienda por qué se recomienda cada película.

Enfoque XAI para tu recomendador:
Tu recomendador usa similitud del coseno entre géneros. Podemos explicar cuáles géneros compartidos son responsables de que dos películas sean similares.


```



In [ ]:
## inteligencia Atificial explicable PARA EL SISTEMA DE RECOMENDACION BASADO EN CONTENIDO.

In [ ]:
def explicar_recomendacion(pelicula_id_base, pelicula_id_recomendada, peliculas_df):
    # Extraer vectores de géneros (0s y 1s)
    base = peliculas_df.loc[pelicula_id_base].drop('titulo')
    recomendada = peliculas_df.loc[pelicula_id_recomendada].drop('titulo')

    # Comparar géneros (element-wise AND)
    generos_comunes = (base & recomendada)

    # Filtrar solo los géneros en común
    generos_activos = generos_comunes[generos_comunes == 1].index.tolist()

    return generos_activos


In [ ]:
def recomendar_peliculas_xai(movie_title, peliculas_df, similitud_df, top_n=4):
    # Buscar ID
    pelicula_id = peliculas_df[peliculas_df['titulo'].str.lower() == movie_title.lower()].index

    if pelicula_id.empty:
        print("Película no encontrada.")
        return

    pelicula_id = pelicula_id[0]

    # Calcular similitudes
    similares = similitud_df[pelicula_id].sort_values(ascending=False).drop(pelicula_id)

    # Seleccionar las top N
    recomendadas = similares.head(top_n)

    print(f"\n🎬 Películas similares a: {peliculas_df.loc[pelicula_id, 'titulo']}\n")

    for i, (rec_id, score) in enumerate(recomendadas.items(), 1):
        titulo = peliculas_df.loc[rec_id, 'titulo']
        generos_comunes = explicar_recomendacion(pelicula_id, rec_id, peliculas_df)
        print(f"{i}. {titulo} (Similitud: {score:.2f})")
        print(f"   ➤ Géneros en común: {', '.join(generos_comunes) if generos_comunes else 'Ninguno'}\n")


In [ ]:
recomendar_peliculas_xai("Forrest Gump", peliculas_df, similitud_df)



🎬 Películas similares a: Forrest Gump

1. Life Is Beautiful (La Vita è bella) (Similitud: 1.00)
   ➤ Géneros en común: Comedy, Drama, Romance, War

2. I Served the King of England (Obsluhoval jsem anglického krále) (Similitud: 1.00)
   ➤ Géneros en común: Comedy, Drama, Romance, War

3. Train of Life (Train de vie) (Similitud: 1.00)
   ➤ Géneros en común: Comedy, Drama, Romance, War

4. Tiger and the Snow, The (La tigre e la neve) (Similitud: 1.00)
   ➤ Géneros en común: Comedy, Drama, Romance, War



In [ ]:
def metricas_coverage(movie_title, peliculas_df, similitud_df, top_n=4):
    pelicula_id = peliculas_df[peliculas_df['titulo'].str.lower() == movie_title.lower()].index
    if pelicula_id.empty:
        return None
    pelicula_id = pelicula_id[0]
    similares = similitud_df[pelicula_id].sort_values(ascending=False).drop(pelicula_id).head(top_n)

    con_genero_comun = 0
    for rec_id in similares.index:
        generos_comunes = explicar_recomendacion(pelicula_id, rec_id, peliculas_df)
        if generos_comunes:
            con_genero_comun += 1

    cobertura = con_genero_comun / top_n
    print(f"✅ Cobertura de explicación: {cobertura * 100:.1f}%")



In [ ]:
metricas_coverage("Forrest Gump", peliculas_df, similitud_df)

✅ Cobertura de explicación: 100.0%


###**FLUJO DE TRABAJO AUTOMATIZADO CON PIPELINES**
------



Objetivo general del sistema
Crear un recomendador de películas basado en los géneros de las películas (recomendación basada en contenido), utilizando un flujo de trabajo automatizado con pipelines para que sea más limpio, reutilizable y escalable.



Paso 1: Carga de datos desde SQL
¿Qué hiciste?
Cargaste los datos de las dos tablas (movies y ratings) desde una base de datos relacional usando pandas.read_sql_query().

¿Objetivo?
Tener los datos actualizados directamente desde la base de datos, listos para su procesamiento.

In [ ]:
import pandas as pd
import sqlite3

# Conectar a la base de datos SQLite
conexion = sqlite3.connect('/content/drive/MyDrive/marketing/Marketing/data/db_movies')

# Cargar las tablas de la base de datos
movies = pd.read_sql_query("SELECT * FROM movies", conexion)
ratings = pd.read_sql_query("SELECT * FROM ratings", conexion)

# Mostrar las primeras filas de cada tabla



 Paso 2: Selección y preparación de características
¿Qué hiciste?
Seleccionaste las columnas de géneros y eliminaste columnas innecesarias como 'title' (que no puede convertirse a número para el cálculo de similitud).

¿Objetivo?
Preparar una matriz con solo los datos numéricos que representan los géneros, necesaria para calcular similitudes.

In [ ]:
# Suponiendo que la columna 'genres' de la tabla 'movies' tiene los géneros en formato de texto separado por comas
# Convertimos los géneros en variables numéricas (one-hot encoding)

# Crear una nueva columna para cada género (si no existe)
genres = movies['genres'].str.split('|', expand=True).stack().unique()
for genre in genres:
    movies[genre] = movies['genres'].apply(lambda x: 1 if genre in x else 0)

# Ahora tenemos una columna por cada género de película
movies.head()


,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Paso 3: Cálculo de similitud
¿Qué hiciste?
Calculaste la similitud entre películas usando la similitud del coseno (cosine_similarity) sobre la matriz de géneros.

¿Objetivo?
Crear una matriz que permita saber qué tan parecidas son las películas entre sí, lo cual es esencial para recomendar.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Seleccionamos las columnas de géneros
genre_columns = genres.tolist()

# Extraemos los datos de géneros
X = movies[genre_columns]

# Calculamos la matriz de similitud de coseno
similarity_matrix = cosine_similarity(X)

# Convertimos la matriz de similitud en un DataFrame para que sea más fácil de leer
similarity_df = pd.DataFrame(similarity_matrix, index=movies['title'], columns=movies['title'])
similarity_df.head()


title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),1.000000,0.774597,0.316228,0.258199,0.447214,0.0,0.316228,0.632456,0.0,0.258199,...,0.447214,0.316228,0.316228,0.447214,0.0,0.670820,0.774597,0.00000,0.316228,0.447214
Jumanji (1995),0.774597,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.816497,0.0,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.0,0.288675,0.333333,0.00000,0.000000,0.000000
Grumpier Old Men (1995),0.316228,0.000000,1.000000,0.816497,0.707107,0.0,1.000000,0.000000,0.0,0.000000,...,0.353553,0.000000,0.500000,0.000000,0.0,0.353553,0.408248,0.00000,0.000000,0.707107
Waiting to Exhale (1995),0.258199,0.000000,0.816497,1.000000,0.577350,0.0,0.816497,0.000000,0.0,0.000000,...,0.288675,0.408248,0.816497,0.000000,0.0,0.288675,0.333333,0.57735,0.000000,0.577350
Father of the Bride Part II (1995),0.447214,0.000000,0.707107,0.577350,1.000000,0.0,0.707107,0.000000,0.0,0.000000,...,0.500000,0.000000,0.707107,0.000000,0.0,0.500000,0.577350,0.00000,0.000000,1.000000


Paso 4: Creación de un Pipeline y recomendador
¿Qué hiciste?
Construiste una función de recomendación y envolviste el flujo en un Pipeline (de sklearn) para automatizar los pasos de transformación y cálculo.

¿Objetivo?
Reutilizar el flujo completo fácilmente, aplicar mantenimiento, y facilitar integración con un sistema más grande (por ejemplo, usando MLOps).

In [ ]:
import numpy as np

# Función para recomendar películas similares
def recomendar_peliculas(movie_title, movies, similarity_matrix, top_n=5):
    # Buscar el ID de la película
    pelicula_id = movies[movies['title'].str.lower() == movie_title.lower()].index

    if len(pelicula_id) == 0:
        print("Película no encontrada.")
        return

    pelicula_id = pelicula_id[0]

    # Obtener las similitudes
    similares = similarity_matrix[pelicula_id].argsort()[::-1]  # Ordenar las películas por similitud

    # Excluir la misma película
    similares = similares[similares != pelicula_id]

    # Obtener los títulos de las películas similares
    recomendadas = movies.iloc[similares[:top_n]]['title']

    print(f"\n🎬 Películas similares a: {movie_title}\n")
    for i, titulo in enumerate(recomendadas, 1):
        print(f"{i}. {titulo}")

# Ejemplo de recomendación
recomendar_peliculas("Toy Story (1995)", movies, similarity_matrix)



🎬 Películas similares a: Toy Story (1995)

1. Toy Story 2 (1999)
2. Adventures of Rocky and Bullwinkle, The (2000)
3. Monsters, Inc. (2001)
4. Emperor's New Groove, The (2000)
5. Moana (2016)
